In [22]:
import json
import os 

from typing import Annotated

from dotenv import load_dotenv

from IPython.display import display, HTML
from tmdbv3api import TMDb, Genre
from openai import AsyncOpenAI
from tmdbv3api import TMDb
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function
from tmdbv3api import TMDb, Discover
from tmdbv3api import Movie
import pandas as pd
from datetime import datetime
from tmdbv3api import TMDb, Discover
import os, json
from tmdbv3api import TMDb, Genre, Discover
import semantic_kernel as sk

load_dotenv()

True

In [15]:
tmdb = TMDb()
tmdb.api_key=

In [23]:
os.environ.get('TMDB_API_KEY')

'None'

In [5]:
# Initialize TMDb with your API key
tmdb = TMDb()
tmdb.api_key ='2fa30f6a1d22eb80c6dc9cac9cc67bdc'
# Fetch the official list of movie genres
genre_client = Genre()
all_genres = genre_client.movie_list()

# Build a name→ID lookup (lowercased keys)
GENRE_MAP = {g.name.lower(): g.id for g in all_genres}

# Create a Discover instance for querying movies
discover = Discover()

client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

In [4]:
def recommend_movies(
    genre_id: int,
    preference: str = "newer",
    top_n: int    = 5,
    weight_pop: float = 0.5,
    weight_vote: float = 0.5
):
    """
    Fetch page 1 for `genre_id`, score by weighted(popularity, vote_average),
    filter by age (≤5 yrs new vs >5 yrs old), return top_n dicts.
    """
    current_year = datetime.now().year
    raw = discover.discover_movies({
        "with_genres": genre_id,
        "sort_by":     "popularity.desc",
        "page":        1
    })
    scored = []
    for m in raw:
        try:
            year = int(m.release_date[:4])
        except:
            continue
        age = current_year - year
        if preference == "newer" and age > 5:  continue
        if preference == "older" and age <= 5: continue
        score = weight_pop * m.popularity + weight_vote * m.vote_average
        scored.append({"title": m.title, "year": year, "score": score})
    return sorted(scored, key=lambda x: x["score"], reverse=True)[:top_n]

In [5]:
class MoviePlugin:
    @kernel_function(description="Recommend movies by genre (name or ID), date preference, and number of results")
    def recommend(
        self,
        genre: str,
        preference: str = "newer",
        top_n: str    = "5"
    ) -> str:
        if genre.isdigit():
            gid = int(genre)
        else:
            gid = GENRE_MAP.get(genre.lower())
            if gid is None:
                return (
                    f"Unknown genre '{genre}'. "
                    f"Available: {', '.join(GENRE_MAP.keys())}"
                )
        try:
            n = int(top_n)
        except:
            n = 5

        recs = recommend_movies(genre_id=gid, preference=preference, top_n=n)

        return "\n".join(
            f"{i+1}. {m['title']} ({m['year']}) — score={m['score']:.2f}"
            for i, m in enumerate(recs)
        )

In [6]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[MoviePlugin()],
    name="MovieAgent",
    instructions="You will identify the preferred genres that the customer wants based on their prompt",
)


In [7]:
user_inputs = [
    "I like comedy movies recommend me some.",
    "I don't like that one, please show me horror.",
]

async def main():
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()

In [6]:
GENRES_TO_USE = list(GENRE_MAP.keys())
# 1) Validate that each chosen genre exists in GENRE_MAP
missing = [g for g in GENRES_TO_USE if g not in GENRE_MAP]
if missing:
    raise ValueError(f"These genres are not in GENRE_MAP: {missing}")

# 2) Build a dict of name → ID for quick lookup later
SELECTED_GENRES = {g: GENRE_MAP[g] for g in GENRES_TO_USE}

# 3) Print out your selections for confirmation
print("=== Quiz Genres ===")
for name, gid in SELECTED_GENRES.items():
    print(f" • {name.title():<8} → ID = {gid}")
print("===================")


=== Quiz Genres ===
 • Action   → ID = 28
 • Adventure → ID = 12
 • Animation → ID = 16
 • Comedy   → ID = 35
 • Crime    → ID = 80
 • Documentary → ID = 99
 • Drama    → ID = 18
 • Family   → ID = 10751
 • Fantasy  → ID = 14
 • History  → ID = 36
 • Horror   → ID = 27
 • Music    → ID = 10402
 • Mystery  → ID = 9648
 • Romance  → ID = 10749
 • Science Fiction → ID = 878
 • Tv Movie → ID = 10770
 • Thriller → ID = 53
 • War      → ID = 10752
 • Western  → ID = 37


In [11]:
GENRE_MAP

{'action': 28,
 'adventure': 12,
 'animation': 16,
 'comedy': 35,
 'crime': 80,
 'documentary': 99,
 'drama': 18,
 'family': 10751,
 'fantasy': 14,
 'history': 36,
 'horror': 27,
 'music': 10402,
 'mystery': 9648,
 'romance': 10749,
 'science fiction': 878,
 'tv movie': 10770,
 'thriller': 53,
 'war': 10752,
 'western': 37}

# ─── Step 1: Prompt for up to 5 genres ──────────────────────────────────────

# 1A) Show available genres
available = sorted(GENRE_MAP.keys())
print("Available genres:")
print(", ".join(name.title() for name in available))

# 1B) Prompt until they pick 1–5 valid genres
while True:
    resp = input(
        "\nEnter up to 5 genres (comma-separated), e.g.\n"
        "Action, Comedy, Drama\n→ "
    )
    picks = [g.strip().lower() for g in resp.split(",")]
    # keep only valid and dedupe
    picks = list(dict.fromkeys([g for g in picks if g in GENRE_MAP]))
    if 1 <= len(picks) <= 5:
        break
    print(f"❌ You chose {len(picks)} valid genres; please pick between 1 and 5 from the list above.")

# 1C) Build name→ID map for those picks
SELECTED_GENRES = {g: GENRE_MAP[g] for g in picks}

# 1D) Confirm selection
print("\nSelected genres:")
for name, gid in SELECTED_GENRES.items():
    print(f" • {name.title():<8} → ID {gid}")


In [7]:
import json
import random
from semantic_kernel.functions import kernel_function
from tmdbv3api import Discover, Movie

# Assumes you’ve already done:
#   tmdb = TMDb()
#   tmdb.api_key = YOUR_TMDB_API_KEY
discover = Discover()
movie_api = Movie()

class MovieTinderPlugin:
    @kernel_function(
        description="Step 2: Given genres JSON, return the top 2 movie titles, IDs & years per genre"
    )
    def fetch_options(self, genres_json: str) -> str:
        # 1) Parse the incoming genres list
        try:
            genres = json.loads(genres_json) if isinstance(genres_json, str) else genres_json
        except json.JSONDecodeError:
            genres = [genres_json]
        # 2) For each genre name, look up its ID and fetch top 2 by popularity
        result: dict[str, list[dict]] = {}
        for g in genres:
            gid = GENRE_MAP.get(g.lower())
            if not gid:
                continue
            raw = list(discover.discover_movies({
                "with_genres": gid,
                "sort_by":     "popularity.desc",
                "page":        1
            }))[:2]
            result[g.lower()] = [
                {
                    "id":           m.id,
                    "title":        m.title,
                    "release_year": (m.release_date or "").split("-")[0]
                }
                for m in raw
            ]
        return json.dumps(result)

    @kernel_function(
        description="Step 3: Given options JSON, pick one movie ID per genre"
    )
    def quiz_preferences(self, options_json: str) -> str:
        opts = json.loads(options_json)
        # auto‐pick the first movie for each genre for this prototype
        picks: list[int] = []
        for movies in opts.values():
            if isinstance(movies, list) and movies:
                picks.append(movies[0]["id"])
        return json.dumps(picks)

    @kernel_function(
        description="Step 4: Given picked IDs JSON, build a 25‐movie candidate pool per pick"
    )
    def build_candidates(self, picks_json: str) -> str:
        # 1) parse or coerce into Python list
        try:
            parsed = json.loads(picks_json)
        except json.JSONDecodeError:
            parsed = [int(picks_json)]
        # 2) flatten if it came in as dict
        if isinstance(parsed, dict):
            raw_ids: list[int] = []
            for v in parsed.values():
                raw_ids.extend(v if isinstance(v, list) else [v])
        elif isinstance(parsed, list):
            raw_ids = parsed
        else:
            raise ValueError(f"Invalid picks_input: {picks_json!r}")

        # 3) fetch top 25 popular movies for each pick
        pool: dict[str, list[dict]] = {}
        for pid in raw_ids:
            movies = list(discover.discover_movies({
                "sort_by": "popularity.desc",
                "page":    1
            }))[:25]
            pool[str(pid)] = [
                {
                    "id":           m.id,
                    "title":        m.title,
                    "release_year": (m.release_date or "").split("-")[0]
                }
                for m in movies
            ]
        return json.dumps(pool)

    @kernel_function(
        description="Step 5: From picks and candidates JSON, return 5 full movie recommendations"
    )
    def recommend(self, picks_json: str, candidates_json: str) -> str:
        picks = json.loads(picks_json)
        candidates = json.loads(candidates_json)
        # flatten & unique
        all_ids = []
        for cid_list in (candidates.values() if isinstance(candidates, dict) else []):
            for mid in cid_list:
                if mid not in all_ids:
                    all_ids.append(mid)
        # sample up to 5
        chosen = random.sample(all_ids, k=min(5, len(all_ids)))

        # fetch details for each
        recs: list[dict] = []
        for mid in chosen:
            m = movie_api.details(mid)
            recs.append({
                "id":           m.id,
                "title":        m.title,
                "release_year": (m.release_date or "").split("-")[0],
                "overview":     m.overview or "",
                "popularity":   m.popularity,
                "vote_average": m.vote_average,
            })
        return json.dumps(recs)


In [8]:
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

In [9]:
# 1) Create the Kernel
from semantic_kernel import Kernel
kernel = Kernel()

# 2) Register your MovieTinderPlugin under the namespace "MovieTinder"
#    (this makes its @kernel_function methods available as MovieTinder.fetch_options, etc.)
kernel.add_plugin(
    MovieTinderPlugin(),
    plugin_name="MovieTinder"
)

# 3) Build your ChatCompletionAgent as before
from semantic_kernel.agents import ChatCompletionAgent

agent = ChatCompletionAgent(
    service = chat_completion_service,
    plugins = [MovieTinderPlugin()],
    name    = "MovieAgent",
    instructions = """
You are MovieAgent. To answer any movie request, follow exactly:

1) CALL_FUNCTION: MovieTinder.fetch_options(genres='<JSON list of genres>')
2) Then CALL_FUNCTION: MovieTinder.quiz_preferences(options='<that JSON>')
3) Then CALL_FUNCTION: MovieTinder.build_candidates(picks='<that JSON>')
4) Finally CALL_FUNCTION: MovieTinder.recommend(picks='<step3 JSON>', candidates='<step4 JSON>')

Return only the final JSON array of 5 full movie recommendation objects
(each with id, title, release_year, overview, popularity, vote_average), and nothing else.
"""
)



In [10]:
from semantic_kernel.agents import ChatHistoryAgentThread

thread: ChatHistoryAgentThread | None = None

print("🎬 Welcome to MovieTinder!\n")

# ─── STEP 1: Ask for genres ────────────────────────────────────────────────
print("Agent: Which genres are you in the mood for today? (e.g. Action, Comedy)")
user_msg = input()  # e.g. Action, Comedy

# Stream the agent’s first response (should CALL fetch_options)
async for resp in agent.invoke_stream(messages=user_msg, thread=thread):
    thread = resp.thread
    for item in resp.items:
        if hasattr(item, "text") and item.text:
            print(item.text, end="", flush=True)
        elif hasattr(item, "function_name"):
            print(f"\n→ Calling {item.function_name}({item.arguments})\n")
        elif hasattr(item, "result"):
            print(f"\n← Function result:\n{item.result}\n")
print()  # blank line

# ─── STEPS 2–5: quiz & recommend ───────────────────────────────────────────
while True:
    # The agent will have printed something like “1) Foo vs. Bar → ”
    choice = input()  # type “1” or “2” and press Enter
    async for resp in agent.invoke_stream(messages=choice, thread=thread):
        thread = resp.thread
        for item in resp.items:
            if hasattr(item, "text") and item.text:
                print(item.text, end="", flush=True)
            elif hasattr(item, "function_name"):
                print(f"\n→ Calling {item.function_name}({item.arguments})\n")
            elif hasattr(item, "result"):
                print(f"\n← Function result:\n{item.result}\n")
    print()  # blank line

    # Break once the agent has returned a JSON array of recommendations
    final_text = "".join(
        it.text for it in resp.items
        if hasattr(it, "text") and it.text
    ).strip()
    if final_text.startswith("[") and final_text.endswith("]"):
        break


🎬 Welcome to MovieTinder!

Agent: Which genres are you in the mood for today? (e.g. Action, Comedy)


Function failed. Error: 'list' object has no attribute 'values'
Error invoking function MovieTinderPlugin-quiz_preferences: 'list' object has no attribute 'values'.
Traceback (most recent call last):
  File "/Users/mendoza/Desktop/personal_dev/MSFT-Hackathon/mtind_venv/lib/python3.11/site-packages/semantic_kernel/kernel.py", line 423, in _inner_auto_function_invoke_handler
    result = await context.function.invoke(context.kernel, context.arguments)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mendoza/Desktop/personal_dev/MSFT-Hackathon/mtind_venv/lib/python3.11/site-packages/semantic_kernel/functions/kernel_function.py", line 258, in invoke
    raise e
  File "/Users/mendoza/Desktop/personal_dev/MSFT-Hackathon/mtind_venv/lib/python3.11/site-packages/semantic_kernel/functions/kernel_function.py", line 250, in invoke
    await stack(function_context)
  File "/Users/mendoza/Desktop/personal_dev/MSFT-Hackathon/mtind_venv/lib/python3.11/site-

Function failed. Error: 'list' object has no attribute 'values'
Error invoking function MovieTinderPlugin-quiz_preferences: 'list' object has no attribute 'values'.
Traceback (most recent call last):
  File "/Users/mendoza/Desktop/personal_dev/MSFT-Hackathon/mtind_venv/lib/python3.11/site-packages/semantic_kernel/kernel.py", line 423, in _inner_auto_function_invoke_handler
    result = await context.function.invoke(context.kernel, context.arguments)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mendoza/Desktop/personal_dev/MSFT-Hackathon/mtind_venv/lib/python3.11/site-packages/semantic_kernel/functions/kernel_function.py", line 258, in invoke
    raise e
  File "/Users/mendoza/Desktop/personal_dev/MSFT-Hackathon/mtind_venv/lib/python3.11/site-packages/semantic_kernel/functions/kernel_function.py", line 250, in invoke
    await stack(function_context)
  File "/Users/mendoza/Desktop/personal_dev/MSFT-Hackathon/mtind_venv/lib/python3.11/site-

NameError: name 'resp' is not defined